In [1]:
# ------------------单词切分---------------------
# %% 
import jieba
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
data_path = "./clean_chat_corpus/" # 数据集路径
train_data_name = "xiaohuangji.tsv"

In [3]:
train_data = pd.read_csv(data_path + train_data_name, sep = '\t', header = None)
# エラー出た
# 结巴分词出现AttributeError: 'float' object has no attribute 'decode'错误
# https://my.oschina.net/u/4336279/blog/3569965
train_data = train_data.astype(str) 

In [4]:

# 去除标点符号
# 来源：https://github.com/fxsjy/jieba/issues/169#issuecomment-49504512
punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒
﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠
々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻
︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')
# 对str/unicode
filterpunt = lambda input_str: ''.join(filter(lambda x: x not in punct, input_str))
def sentence_process(input_str):
    '''
    将传入的中文句子调用jieba分词
    返回使用空格拼接的句子
    '''
    return " ".join(list(jieba.cut(filterpunt(input_str))))

In [5]:
# qaa_list = [] # 问题答案list
# for i in range(len(train_data[0])):
#     qaa_list.append([train_data[0][i], train_data[1][i]])

In [6]:
q_list = train_data[0].tolist() # 问题list
a_list = train_data[1].tolist() # 答案list

In [7]:
q_space_list = [sentence_process(i) for i in q_list]


Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/rz/wc4p7dpn0r9dd63vv2vg211h0000gn/T/jieba.cache
Loading model cost 0.413 seconds.
Prefix dict has been built succesfully.


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [9]:
# 修改可以识别单个单词 https://blog.csdn.net/blmoistawinde/article/details/80816179
tv = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", max_df = 0.6, stop_words=None)

In [10]:
train_v = tv.fit_transform(q_space_list) # 使用问题集训练


In [11]:
# dic_index = {}
# space_list = q_space_list[0:5]
# len_list = len(space_list)
# for index in range(len_list):
#     for word in space_list[index].split():
#         if word not in dic_index:
#             dic_index[word] = [index]
#         else:
#             dic_index[word].append(index)
# # %%
# dic_index["你好"]
# # %%
# q_space_list[355]

In [12]:
dic_index = {}
len_list = len(q_space_list)
str_len_list = str(len_list)
for index in range(len_list):
    print("创建倒排表： " + str(index) + "/" + str_len_list)
    for word in q_space_list[index].split():
        if word not in dic_index.keys(): # 单词不在倒排表中
            dic_index[word] = [index]
        else:
            dic_index[word].append(index)


/451512
创建倒排表： 450560/451512
创建倒排表： 450561/451512
创建倒排表： 450562/451512
创建倒排表： 450563/451512
创建倒排表： 450564/451512
创建倒排表： 450565/451512
创建倒排表： 450566/451512
创建倒排表： 450567/451512
创建倒排表： 450568/451512
创建倒排表： 450569/451512
创建倒排表： 450570/451512
创建倒排表： 450571/451512
创建倒排表： 450572/451512
创建倒排表： 450573/451512
创建倒排表： 450574/451512
创建倒排表： 450575/451512
创建倒排表： 450576/451512
创建倒排表： 450577/451512
创建倒排表： 450578/451512
创建倒排表： 450579/451512
创建倒排表： 450580/451512
创建倒排表： 450581/451512
创建倒排表： 450582/451512
创建倒排表： 450583/451512
创建倒排表： 450584/451512
创建倒排表： 450585/451512
创建倒排表： 450586/451512
创建倒排表： 450587/451512
创建倒排表： 450588/451512
创建倒排表： 450589/451512
创建倒排表： 450590/451512
创建倒排表： 450591/451512
创建倒排表： 450592/451512
创建倒排表： 450593/451512
创建倒排表： 450594/451512
创建倒排表： 450595/451512
创建倒排表： 450596/451512
创建倒排表： 450597/451512
创建倒排表： 450598/451512
创建倒排表： 450599/451512
创建倒排表： 450600/451512
创建倒排表： 450601/451512
创建倒排表： 450602/451512
创建倒排表： 450603/451512
创建倒排表： 450604/451512
创建倒排表： 450605/451512
创建倒排表： 450606/451512
创建倒排表

In [13]:
def get_index(str_list, dic_index, threshold=1): # Threshold代表限制同时出现多少个单词的情况返回
    """
    根据输入的list
    和给定的字典倒排表
    输出所在的index
    """
    input_str = str_list[0]
    tmp_list = []
    for word in input_str.split():
        # print("正在查找单词："+word)
        if word in dic_index:
            for index in dic_index[word]:
                tmp_list.append(index)
    if(len(tmp_list)>10000):
        if(threshold > 1):
            len_word_list = len(input_str.split())
            threshold = threshold if threshold <= len_word_list else len_word_list
            tmp_list_choice = {}
            for item in tmp_list:
                if item in tmp_list_choice:
                    tmp_list_choice[item] += 1
                else:
                    tmp_list_choice[item] = 1
            tmp_list = []
            for k, v in tmp_list_choice.items():
                if v >= threshold:
                    tmp_list.append(k)
    return list(set(tmp_list))


In [14]:
def get_best_index_list(input_str):
    input_content = [sentence_process(input_str)]

    test_v = tv.transform(input_content)

    test_array = test_v.toarray()
    
    # tmp_cosine_sim = []
    # possible_index_list = get_index(input_content, dic_index, 3)
    # len_possible_index_list = len(possible_index_list)
    # str_len_possible_index_list = str(len_possible_index_list)
    # for index in range(len_possible_index_list):
    #     print("当前进度："+str(index)+" / "+str_len_possible_index_list)
    #     tmp_cosine_sim.append([float(cosine_similarity(train_v[possible_index_list[index]], test_array)), possible_index_list[index]])

    tmp_cosine_sim = []
    possible_index_list = get_index(input_content, dic_index, 1)
    len_possible_index_list = len(possible_index_list)
    for index in range(len_possible_index_list):
        tmp_cosine_sim.append([float(cosine_similarity(train_v[possible_index_list[index]], test_array)), possible_index_list[index]])

    # 排序筛选出来的内容
    tmp_return = sorted(tmp_cosine_sim, key=lambda x: x[0], reverse=True)

    return tmp_return

In [15]:
def get_qalist_with_list(input_str, range_int=1, output_q=False):
    tmp_return = get_best_index_list(input_str)
    result_list_len = len(tmp_return)
    if result_list_len != 0:
        range_int = result_list_len if result_list_len < range_int else range_int
        for i in range(range_int):
            if(output_q):print(str(i+1)+". 最匹配的问题："+q_list[tmp_return[i][1]])
            print(str(i+1)+". 其对应答案为："+a_list[tmp_return[i][1]])
    else:
        print("你在说些什么，本宝宝听不太懂呢")

In [19]:
get_qalist_with_list("我喜欢你",5,True)

进度：8905 / 10005
当前进度：8906 / 10005
当前进度：8907 / 10005
当前进度：8908 / 10005
当前进度：8909 / 10005
当前进度：8910 / 10005
当前进度：8911 / 10005
当前进度：8912 / 10005
当前进度：8913 / 10005
当前进度：8914 / 10005
当前进度：8915 / 10005
当前进度：8916 / 10005
当前进度：8917 / 10005
当前进度：8918 / 10005
当前进度：8919 / 10005
当前进度：8920 / 10005
当前进度：8921 / 10005
当前进度：8922 / 10005
当前进度：8923 / 10005
当前进度：8924 / 10005
当前进度：8925 / 10005
当前进度：8926 / 10005
当前进度：8927 / 10005
当前进度：8928 / 10005
当前进度：8929 / 10005
当前进度：8930 / 10005
当前进度：8931 / 10005
当前进度：8932 / 10005
当前进度：8933 / 10005
当前进度：8934 / 10005
当前进度：8935 / 10005
当前进度：8936 / 10005
当前进度：8937 / 10005
当前进度：8938 / 10005
当前进度：8939 / 10005
当前进度：8940 / 10005
当前进度：8941 / 10005
当前进度：8942 / 10005
当前进度：8943 / 10005
当前进度：8944 / 10005
当前进度：8945 / 10005
当前进度：8946 / 10005
当前进度：8947 / 10005
当前进度：8948 / 10005
当前进度：8949 / 10005
当前进度：8950 / 10005
当前进度：8951 / 10005
当前进度：8952 / 10005
当前进度：8953 / 10005
当前进度：8954 / 10005
当前进度：8955 / 10005
当前进度：8956 / 10005
当前进度：8957 / 10005
当前进度：8958 / 10005
当前进度：8959 / 10005
当前进度：8960 / 